This is a live notebook with experimental code to develop analysis investigating the correlation between features.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import configparser

import os
print(os.listdir("."))

# import scipy.stats as spstats
# # fourier transform
# from scipy.fft import fft, ifft

# from sklearn.metrics import classification_report
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# import matplotlib.pyplot as plt
# import pandas as pd
# import seaborn as sns
# # from statsmodels.tsa.api import acf, graphics, pacf
# from statsmodels.tsa.ar_model import AutoReg
# # from statsmodels.tsa.ar_model import ar_select_order


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'analyse_results.ipynb', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'feature_analysis.ipynb', 'test_sample_entropy_implementations.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'processing_status.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH
from predicament.utils.file_utils import drop_nan_cols
from predicament.utils.file_utils import load_dataframe_and_config
from predicament.utils.file_utils import write_dataframe_and_config
from predicament.utils.config_parser import config_to_dict

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import IDEAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from predicament.data.features import filter_features
from predicament.data.features import derive_feature_types

In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

## dreem (EEG) data or E4 data



In [6]:
subdirs = ['dreem_4secs', 'dreem_10secs', 'E4_4secs', 'E4_10secs']

config = configparser.ConfigParser()

subdir_feature_types = {}
subdir_label_mappings = {}
combined_types = set()
combined_conditions = set()

for subdir in subdirs: 
    print(f"subdir = {subdir}")
    data_dir = os.path.join(FEATURED_BASE_PATH,subdir)
    config_path = os.path.join(data_dir, 'details.cfg')
    with open(config_path, 'r') as config_file:
        config.read_file(config_file)
# convert config to dictionary
    dict_config =  config_to_dict(config)
    feature_names = dict_config['FEATURED']['feature_names']
    feature_set = dict_config['FEATURED']['feature_set']
    label_cols = dict_config['WINDOWED']['label_cols']
    feature_types = derive_feature_types(feature_names, feature_set, label_cols)
    print(f"non-ideal features: {set(feature_set)-IDEAL_FEATURE_GROUP}")
    print(f"missing ideal features: {IDEAL_FEATURE_GROUP-set(feature_set)}")
    subdir_feature_types[subdir] = feature_types
    combined_types.update(feature_types)
    label_mapping = dict_config['LOAD']['label_mapping']
    subdir_label_mappings[subdir] = label_mapping
    combined_conditions.update(label_mapping)
    print()

subdir = dreem_4secs
non-ideal features: {'LempelZivEntropy', 'Energy', 'FreqSkewness'}
missing ideal features: {'HRVRMSSD'}

subdir = dreem_10secs
non-ideal features: {'LempelZivEntropy', 'Energy', 'FreqSkewness'}
missing ideal features: {'HRVRMSSD'}

subdir = E4_4secs
non-ideal features: {'LempelZivEntropy', 'FreqSkewness'}
missing ideal features: set()

subdir = E4_10secs
non-ideal features: {'LempelZivEntropy', 'FreqSkewness'}
missing ideal features: set()



In [7]:
print("Comparing each subdir with combined features from all:\n\n")
print(f"Combined is {';'.join(combined_types)}\n\n")
for subdir in subdirs:
    diff = combined_types - set(subdir_feature_types[subdir])
    print(f"{subdir} is missing:\n\t{diff}")

Comparing each subdir with combined features from all:


Combined is Mean;Hurst_H;LempelZivEntropy[b=1];IQR;Max;Min;LempelZivComplexity[b=8];Hurst_C;MeanFreq;MAD;LempelZivComplexity[b=4];SD;FreqSkewness;SampleEntropy[m=2];arCoeff;Correlation;LyapunovExponent;HRVRMSSD[Normed];LempelZivEntropy[b=4];FreqKurtosis;LempelZivComplexity[b=2];Energy;MaxFreqInd;LempelZivComplexity[b=1]


dreem_4secs is missing:
	{'HRVRMSSD[Normed]'}
dreem_10secs is missing:
	{'HRVRMSSD[Normed]', 'LempelZivEntropy[b=4]'}
E4_4secs is missing:
	{'Energy', 'LempelZivEntropy[b=4]'}
E4_10secs is missing:
	{'Energy', 'LempelZivEntropy[b=4]'}


In [8]:
print("Comparing each subdir with combined conditions from all:")
print(f"Combined is {combined_conditions}")
for subdir in subdirs:
    diff = combined_conditions - set(subdir_label_mappings[subdir])
    print(f"{subdir} is missing:\n\t{diff}")

Comparing each subdir with combined conditions from all:
Combined is {'familiar_music', 'tchaikovsky', 'family_inter', 'wildlife_video', 'exper_video'}
dreem_4secs is missing:
	set()
dreem_10secs is missing:
	set()
E4_4secs is missing:
	set()
E4_10secs is missing:
	set()
